<a href="https://colab.research.google.com/github/stereifberger/logical_derivations_with_transformers/blob/main/implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone whole github repository with necessary files
!git clone https://github.com/stereifberger/logical_derivations_with_transformers
!cd /content/logical_derivations_with_transformers

Cloning into 'logical_derivations_with_transformers'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 58 (delta 23), reused 46 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 110.39 KiB | 15.77 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [2]:
# Import libraries
import sys
sys.path.append('logical_derivations_with_transformers')
from imports import *

In [ ]:
importlib.reload(train_test)

<module 'train_test' from '/home/str/logical_derivations_with_transformers/train_test.py'>

In [3]:
# Set random seed for reproducibility
seed(42)
torch.manual_seed(42)

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
# Prepare data
total_samples = 200
#batch_size = 32
data_inputs, data_targets, max_input_len, max_target_len = utils.prepare_data(num_samples=total_samples)
t_nu = 5
vocab_size = t_nu + 11  # Number of symbols including PAD, SOS, EOS
inputs_train, inputs_test, targets_train, targets_test = train_test_split(
    data_inputs, data_targets, test_size=0.2, random_state=42
)

# Construct data loader
#train_data = list(zip(inputs_train, targets_train))
#test_data = list(zip(inputs_test, targets_test))
#train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=train_test.collate_batch)
#test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, collate_fn=train_test.collate_batch)

In [7]:
import string

alphabet = string.ascii_lowercase  # Get lowercase letters 'a', 'b', 'c', ...

def convert_to_letters(data):
  """Converts a list of lists of integers to letters.

  Args:
    data: A list of lists of integers.

  Returns:
    A list of lists of letters.
  """
  converted_data = []
  for sublist in data:
    converted_sublist = [alphabet[i] for i in sublist]  # Map integers to letters
    converted_data.append(converted_sublist)
  return converted_data

# Convert the data
inputs_train = convert_to_letters(inputs_train)
inputs_test = convert_to_letters(inputs_test)
targets_train = convert_to_letters(targets_train)
targets_test = convert_to_letters(targets_test)

# Print an example to verify
print(inputs_train[0])  # Print the first element of inputs_train_letters

['h', 'f', 'k', 'e', 'i', 'd', 'g', 'h', 'b', 'l', 'h', 'f', 'k', 'e', 'i', 'i', 'p', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']


In [6]:
# Model parameters
input_dim = vocab_size
output_dim = vocab_size
emb_dim = 128
num_heads = 5
hidden_dim = 256
num_layers = 3
dropout = 0.1

# Check for avaiable GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print("Using CUDA")
else:
    print("Using CPU")

# Initialize the model
encoder = architectures.TransformerEncoder(input_dim, emb_dim, num_heads, hidden_dim, num_layers, dropout, max_seq_len=max_input_len)
decoder = architectures.TransformerDecoder(output_dim, emb_dim, num_heads, hidden_dim, num_layers, dropout, max_seq_len=max_target_len)
model = architectures.Seq2SeqTransformer(encoder, decoder, device).to(device)

Using CUDA


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


# Gemma

In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, Dataset

In [11]:
!pip install transformers accelerate datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "KingNish/Qwen2.5-0.5b-Test-ft"
model = AutoModelForCausalLM.from_pretrained(model_name) # Use AutoModelForCausalLM instead of AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [12]:
# Check for avaiable GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print("Using CUDA")
else:
    print("Using CPU")

Using CUDA


In [39]:
train_dataset = Dataset.from_dict({"input": inputs_train, "target": targets_train})
test_dataset = Dataset.from_dict({"input": inputs_test, "target": targets_test})

In [40]:
# prompt: Preprocess train_dataset and test_dataset

def preprocess_function(examples):
    inputs = ["".join(x) for x in examples["input"]]
    targets = ["".join(x) for x in examples["target"]]
    # Use the same max_length for both input and target
    max_len = max(max_input_len, max_target_len)
    model_inputs = tokenizer(inputs, padding="max_length", max_length=max_len, truncation=True)
    labels = tokenizer(targets, padding="max_length", max_length=max_len, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)

test_dataset = test_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=test_dataset.column_names,
)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [42]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 160
})

In [47]:
training_args = TrainingArguments(
       output_dir="./results",
       per_device_train_batch_size=32,
       per_device_eval_batch_size=32,
       num_train_epochs=20,
       report_to="tensorboard",
   )

In [48]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.04704412460327148, metrics={'train_runtime': 81.431, 'train_samples_per_second': 39.297, 'train_steps_per_second': 1.228, 'total_flos': 666549443788800.0, 'train_loss': 0.04704412460327148, 'epoch': 20.0})

In [37]:
from operator import index
for i in range(min(50, len(test_dataset))):  # Use min to avoid IndexError if test_dataset has fewer than 50 entries
    example = test_dataset[i]
    inputs = example['input_ids']
    labels = example['labels']  # Access labels directly

    # Assuming your model is called 'model' and is already loaded
    with torch.no_grad():
        # Reshape labels to have batch size of 1 and move to device
        outputs = model(labels.unsqueeze(0).to(device)).logits
        # Note: Providing labels as input for teacher forcing during inference

    # Process the model outputs
    predicted_ids = torch.argmax(outputs, dim=-1)
    predicted_tokens = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

    #reverse_inputs = "".join([symb_reverse[i] for i in tokenizer.decode(inputs)])
    reverse_inputs = tokenizer.decode(inputs)
    index = reverse_inputs.find("a")
    reverse_inputs = "".join([symb_reverse[i] for i in reverse_inputs[:index]])

    reverse_target = tokenizer.decode(labels)
    index = reverse_target.find("a")
    reverse_target = "".join([symb_reverse[i] for i in reverse_target[:index]])

    pred = predicted_tokens[0]
    index = pred.find("a")
    reverse_pred = "".join([symb_reverse[i] for i in pred[:index]])

    print(f"Input {i+1}: {reverse_inputs}") # Decode the input sequence
    print(f"Target {i+1}: {reverse_target}")
    print(f"Output {i+1}: {reverse_pred}") # Print the decoded prediction
    print("-" * 20)

Input 1: (((t→r)→t)∧((p∨s)→s))⊢((((t→r)→t)∧((p∨s)→s))∨((q∨p)∧q))
Target 1: (((t→r)→t)∧((p∨s)→s))((((t→r)→t)∧((p∨s)→s))∨((q∨p)∧q))
Output 1: (((→rq)tt((t)∧)∨((((t→rttp∨s((p∨s))t)∨((q∨p)→tq)tt))))))
--------------------
Input 2: (p∧p)⊢((p∧p)→p)
Target 2: (p∧p)p((p∧p)→p)
Output 2: (()()p)()→∧pp)→)p
--------------------
Input 3: (s∧q)p⊢(p→(r∨p))
Target 3: (s∧q)p(r∨p)(p→(r∨p))
Output 3: (s∧q)pq)qpt→()))p)
--------------------
Input 4: ((r∨(t∨p))∨t)⊢(((r∨(t∨p))∨t)∨(q∧s))
Target 4: ((r∨(t∨p))∨t)(((r∨(t∨p))∨t)∨(q∧s))
Output 4: (q)s∧st∧t∨(tt∨(())t∨t∧t)∧p)∧(q∧s)r∨((t)
--------------------
Input 5: ((q∨(t→r))∨(q∧s))⊢(((q∨(t→r))∨(q∧s))∧(p∨((q∨(t→r))∨(q∧s))))
Target 5: ((q∨(t→r))∨(q∧s))(p∨((q∨(t→r))∨(q∧s)))(((q∨(t→r))∨(q∧s))∧(p∨((q∨(t→r))∨(q∧s))))
Output 5: (t→(r→((((q((t→((t→(t→t((q∧s∧s(t→(((t→t(q∧s∧st)∧t→p∨t→(t→t(q(q∧s)∧(((t→t→r)r)
--------------------
Input 6: q⊢(q→(((q→s)→¬t)∨q))
Target 6: q(((q→s)→¬t)∨q)(q→(((q→s)→¬t)∨q))
Output 6: ((((q→s)→q()→(q(s)∨))))())t)∨qq)((t)∨t)∨
--------------------


In [88]:
# Get a batch of inputs from test_dataset
batch = next(iter(test_dataset))
# Reshape inputs to have batch size of 1
inputs = torch.tensor(batch['input_ids']).unsqueeze(0).to(device)
# Generate outputs using the model
outputs = model.generate(inputs)

<ipython-input-88-1c82b601c105>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(batch['input_ids']).unsqueeze(0).to(device)


In [89]:
for i in test_dataset:
  print(i)

{'input_ids': tensor([    73,   7628,    866,     71,  29307,  12441,  43390,    573,  69440,
         69440,  69440,  69440,  69440,  69440,  69440,  69440,  69440,  28458,
            64, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 15

In [90]:
print(outputs)

tensor([[    73,   7628,    866,     71,  29307,  12441,  43390,    573,  69440,
          69440,  69440,  69440,  69440,  69440,  69440,  69440,  69440,  28458,
             64, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151

In [91]:
# Decode the outputs
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the decoded outputs
for output in decoded_outputs:
    print(output)

jdcghhbmfiljdipaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaipaaaaaaaafiljeaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


In [29]:
# Reverse mapping from numerical values to symbols
symb_reverse = {"a": "",
                "b": "p",
                "c": "q",
                "d": "r",
                "e": "s",
                "f": "t",
                "g": "⊢",
                "h": "(",
                "i": ")",
                "j": "¬",
                "k": "→",
                "l": "∨",
                "m": "∧",
                "n": "⊥",
                "o": "",
                "p": "",
                }

In [92]:
output_reversed = "".join([symb_reverse[i] for i in output])
output_reversed

'¬rq⊢((p∧t)∨¬r))t)∨¬s'